# Performance Evaluation of Pre-trained Vision Transformers vs. CNNs for Urdu Handwritten Character Recognition.

#### pre-trained on imagenet

#### Author: Zaryab rahman
#### Date:  28/9/25


### Imports

In [19]:
import os
import time
import json
import logging
import torch
import timm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from torch import nn
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import datasets, transforms
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm.auto import tqdm

In [ ]:
# unzip a file
!unzip "/content/archive (4).zip" -d "/content/uhar/"


### Config

In [20]:
CONFIG = {
    "data_path": "/content/uhar/data/data",

    "num_classes": 43,
    "batch_size": 64,
    "image_size": 224,
    "num_epochs": 50,
    "learning_rate": 0.001,
    "early_stopping_patience": 5,
    "validation_split": 0.15,               # 15% of training data used for validation

    "results_dir": "/content/drive/MyDrive/uhar/results/",
    "checkpoints_dir": "/content/drive/MyDrive/uhar/checkpoints/",

    "models_to_evaluate": [
        # cnn
        "resnet18",
        "resnet50",
        # "vgg16", much larger so avoid for now
        "mobilenetv2_100",
        "efficientnet_b0",

        # vsion transformer
        "vit_tiny_patch16_224",
        "swin_tiny_patch4_window7_224",
        "deit_tiny_distilled_patch16_224",
        "mobilevit_s"
    ]
}

### setup device, logging, and directories



In [21]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

os.makedirs(CONFIG["results_dir"], exist_ok=True)
os.makedirs(CONFIG["checkpoints_dir"], exist_ok=True)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.FileHandler("experiment_log.log"),
                        logging.StreamHandler()
                    ])

logging.info(f"Using device: {DEVICE}")

### EarlyStopping

In [22]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, checkpoint_path='checkpoint.pth'):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.checkpoint_path = checkpoint_path

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            logging.info(f'earlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        logging.info(f'validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving best model...')
        torch.save(model.state_dict(), self.checkpoint_path)
        self.val_loss_min = val_loss

### Data Loading and Verification


In [23]:

def create_dataloaders(config):
    data_path = config["data_path"]
    if not os.path.exists(data_path):
        logging.error(f"Path not exists : {data_path}")
        return None, None, None, []

    data_transform = transforms.Compose([
        transforms.Resize((config["image_size"], config["image_size"])),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dir = "/content/uhar/data/data/characters_train_set"
    test_dir = "/content/uhar/data/data/characters_test_set"

    if not os.path.exists(train_dir) or not os.path.exists(test_dir):
        logging.error("Training or test directory not found.")
        return None, None, None, []


    full_train_dataset = datasets.ImageFolder(train_dir, transform=data_transform)

    # case incentivate version since there are some inconsities in the naming of the folders

    class_to_idx_lower = {cls.lower(): idx for cls, idx in full_train_dataset.class_to_idx.items()}
    class_names = full_train_dataset.classes

    config["num_classes"] = len(class_names)
    logging.info(f"Established class mapping from training data. Found {len(class_names)} classes.")

    test_dataset = datasets.ImageFolder(test_dir, transform=data_transform)
    logging.info(f"Initially loaded test set with {len(test_dataset)} images.")

    remapped_targets = []
    valid_imgs = []

    for path, original_idx in test_dataset.imgs:
        class_name_from_folder = os.path.basename(os.path.dirname(path)).lower()

        correct_idx = class_to_idx_lower.get(class_name_from_folder)

        if correct_idx is not None:
            remapped_targets.append(correct_idx)
            valid_imgs.append((path, correct_idx))
        else:
            logging.warning(f"Class '{os.path.basename(os.path.dirname(path))}' from test set has no equivalent in training set. Skipping image: {path}")

    test_dataset.imgs = valid_imgs
    test_dataset.samples = valid_imgs
    test_dataset.targets = remapped_targets
    test_dataset.classes = full_train_dataset.classes
    test_dataset.class_to_idx = full_train_dataset.class_to_idx

    logging.info(f"Successfully remapped test dataset. Total valid test images: {len(test_dataset)}")


    val_split = config["validation_split"]
    dataset_size = len(full_train_dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(val_split * dataset_size))
    np.random.seed(42)
    np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    num_workers = 1
    train_loader = DataLoader(full_train_dataset, batch_size=config["batch_size"], sampler=train_sampler, num_workers=num_workers)
    val_loader = DataLoader(full_train_dataset, batch_size=config["batch_size"], sampler=val_sampler, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=num_workers)

    logging.info(f"Data loaders created: {len(train_indices)} train images, {len(val_indices)} validation images, {len(test_dataset)} test images.")

    return train_loader, val_loader, test_loader, class_names

train_loader, val_loader, test_loader, class_names = create_dataloaders(CONFIG)

### Core Training and Evaluation Functions

In [24]:
def train_model(model, model_name, config, train_dataloader, val_dataloader):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

    best_model_path = os.path.join(config["checkpoints_dir"], f"{model_name}_best_model.pth")
    early_stopper = EarlyStopping(patience=config["early_stopping_patience"], checkpoint_path=best_model_path)

    start_time = time.time()
    for epoch in range(config["num_epochs"]):
        model.train()
        train_loss, train_acc = 0, 0
        train_pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1} [Train]", leave=False)
        for X, y in train_pbar:
            X, y = X.to(DEVICE), y.to(DEVICE)
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            train_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            batch_acc = (y_pred_class == y).sum().item() / len(y_pred)
            train_acc += batch_acc
            train_pbar.set_postfix(loss=loss.item(), acc=batch_acc)
        train_loss /= len(train_dataloader)
        train_acc /= len(train_dataloader)

        model.eval()
        val_loss, val_acc = 0, 0
        val_pbar = tqdm(val_dataloader, desc=f"Epoch {epoch+1} [Val]", leave=False)
        with torch.no_grad():
            for X, y in val_pbar:
                X, y = X.to(DEVICE), y.to(DEVICE)
                y_pred = model(X)
                loss = loss_fn(y_pred, y)
                val_loss += loss.item()
                y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
                batch_acc = (y_pred_class == y).sum().item() / len(y_pred)
                val_acc += batch_acc
                val_pbar.set_postfix(loss=loss.item(), acc=batch_acc)
        val_loss /= len(val_dataloader)
        val_acc /= len(val_dataloader)

        logging.info(f"Epoch: {epoch+1} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        early_stopper(val_loss, model)
        if early_stopper.early_stop:
            logging.info("Early stopping triggered.")
            break

    train_time = time.time() - start_time
    logging.info(f"Loading best model weights from epoch checkpoint.")
    model.load_state_dict(torch.load(best_model_path))
    return model, history, train_time

def evaluate_on_test_set(model, test_dataloader, class_names):
    model.eval()
    y_true, y_pred = [], []
    start_time = time.time()
    with torch.no_grad():
        for images, labels in tqdm(test_dataloader, desc="Testing"):
            images = images.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    total_time = time.time() - start_time
    inference_time_ms = (total_time / len(test_dataloader.dataset)) * 1000
    accuracy = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True, zero_division=0)
    cm = confusion_matrix(y_true, y_pred)
    return accuracy, report, cm, inference_time_ms

### Visualization Functions

In [25]:
def plot_training_curves(history, model_name, save_dir):
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
    fig.suptitle(f'Training and Validation Metrics for {model_name}', fontsize=16)

    ax1.plot(history['train_loss'], label='Train Loss')
    ax1.plot(history['val_loss'], label='Validation Loss')
    ax1.set_title('Loss Curves')
    ax1.set_xlabel('Epochs'); ax1.set_ylabel('Loss'); ax1.legend()

    ax2.plot(history['train_acc'], label='Train Accuracy')
    ax2.plot(history['val_acc'], label='Validation Accuracy')
    ax2.set_title('Accuracy Curves')
    ax2.set_xlabel('Epochs'); ax2.set_ylabel('Accuracy'); ax2.legend()

    plt.savefig(os.path.join(save_dir, f"{model_name}_training_curves.png"))
    plt.close()

def plot_confusion_matrix(cm, model_name, class_names, save_dir, normalize=False):
    plt.style.use('default')
    plt.figure(figsize=(16, 14))

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        fmt, title, fname = '.2f', f'Normalized Confusion Matrix: {model_name}', f"{model_name}_cm_normalized.png"
    else:
        fmt, title, fname = 'd', f'Confusion Matrix: {model_name}', f"{model_name}_cm.png"

    sns.heatmap(cm, annot=False, fmt=fmt, cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(title); plt.xlabel('Predicted'); plt.ylabel('True')
    plt.savefig(os.path.join(save_dir, fname))
    plt.close()

### main experiment execution

In [26]:
def run_and_save_single_model(model_name, config, train_loader, val_loader, test_loader, class_names):
    """
    trains,
    evaluates,
    and saves
    the results for a single model.

    """
    logging.info(f"\\n{'='*60}\\nstarting experiment for: {model_name} \\n{'='*60}")

    model = timm.create_model(model_name, pretrained=True, num_classes=config["num_classes"]).to(DEVICE)
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    logging.info(f"Trainable parameters: {num_params / 1e6:.2f}M")

    model, history, train_time = train_model(model, model_name, config, train_loader, val_loader)

    plot_training_curves(history, model_name, config["results_dir"])

    accuracy, report, cm, inference_time = evaluate_on_test_set(model, test_loader, class_names)

    plot_confusion_matrix(cm, model_name, class_names, config["results_dir"], normalize=False)
    plot_confusion_matrix(cm, model_name, class_names, config["results_dir"], normalize=True)

    model_results = {
        "Model": model_name,
        "Type": "ViT" if any(x in model_name for x in ["vit", "swin", "deit"]) else "CNN",
        "Params (M)": round(num_params / 1e6, 2),
        "Train Time (s)": round(train_time, 2),
        "Test Accuracy": round(accuracy, 4),
        "F1-Score (Macro)": round(report["macro avg"]["f1-score"], 4),
        "Inference Time (ms/img)": round(inference_time, 2)
    }

    # Save results to a model-specific JSON file
    result_filepath = os.path.join(CONFIG["results_dir"], f"{model_name}_results.json")
    with open(result_filepath, 'w') as f:
        json.dump(model_results, f, indent=4)

    logging.info(f"Results for {model_name} saved to {result_filepath}")
    logging.info(f"Finished experiment for: {model_name}")

In [ ]:

model_to_run_now = "efficientnet_b0"

if model_to_run_now not in CONFIG["models_to_evaluate"]:
    logging.error(f"error: Model '{model_to_run_now}' not found in CONFIG['models_to_evaluate'].")
    logging.error(f"please choose from: {CONFIG['models_to_evaluate']}")
else:
    if train_loader:
        run_and_save_single_model(model_to_run_now, CONFIG, train_loader, val_loader, test_loader, class_names)
    else:
        logging.error("Data cant be loaded")

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Epoch 1 [Train]:   0%|          | 0/377 [00:00<?, ?it/s]


### **Aggregate Results and Generate Final Summary**
After you have run the experiment for one or more models, run the cell below to gather all the saved results and generate the final comparative plots and summary CSV file.

In [ ]:

import glob

result_files = glob.glob(os.path.join(CONFIG["results_dir"], "*_results.json"))
all_results = []

for f in result_files:
    with open(f, 'r') as file:
        all_results.append(json.load(file))

if all_results:
    model_order = CONFIG["models_to_evaluate"]
    all_results.sort(key=lambda x: model_order.index(x['Model']) if x['Model'] in model_order else -1)

    summary_df = pd.DataFrame(all_results).set_index("Model")

    logging.info("\\n\\n" + "="*80)
    logging.info("**************************************Final Expirement Summary*************************************")
    logging.info("="*80 + "\\n")
    print(summary_df)

    summary_df.to_csv(os.path.join(CONFIG["results_dir"], "performance_summary.csv"))

    plt.style.use('seaborn-v0_8-whitegrid')
    fig, axes = plt.subplots(2, 2, figsize=(20, 15))
    fig.suptitle('Model Performance Comparison', fontsize=20)

    sns.barplot(x=summary_df.index, y='Test Accuracy', hue='Type', data=summary_df, ax=axes[0, 0], dodge=False)
    axes[0, 0].set_title('Test Accuracy'); axes[0, 0].tick_params(axis='x', rotation=45)

    sns.barplot(x=summary_df.index, y='F1-Score (Macro)', hue='Type', data=summary_df, ax=axes[0, 1], dodge=False)
    axes[0, 1].set_title('Macro F1-Score'); axes[0, 1].tick_params(axis='x', rotation=45)

    sns.barplot(x=summary_df.index, y='Inference Time (ms/img)', hue='Type', data=summary_df, ax=axes[1, 0], dodge=False)
    axes[1, 0].set_title('Inference Time per Image (ms)'); axes[1, 0].tick_params(axis='x', rotation=45)

    sns.barplot(x=summary_df.index, y='Params (M)', hue='Type', data=summary_df, ax=axes[1, 1], dodge=False)
    axes[1, 1].set_title('Trainable Parameters (M)'); axes[1, 1].tick_params(axis='x', rotation=45)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(os.path.join(CONFIG["results_dir"], "summary_comparison_plots.png"))
    plt.show()

    logging.info("\\nexperiment summary complete.")
else:
    logging.warning("no result files found in the results directory. Nothing to summarize.")
